# Get redshift data from NED based on the tabulated NED-D file


https://ned.ipac.caltech.edu/Library/Distances/NED26.05.1-D-12.1.0-20160501.csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("NED26.05.1-D-12.1.0-20160501.csv",skiprows=12)

In [3]:
df.head()

,Exclusion Code,D,G,Galaxy ID,m-M,err,D (Mpc),Method,REFCODE,SN ID,redshift (z),Hubble const.,Adopted LMC modulus,Date (Yr. - 1980),Notes
0,RN,1,1,SDSS-II SN 13651,41.64,0.17,1700,SNIa SDSS,2014arXiv1401.3317S,SDSS-II SN 13651,0.249917,70,NaN,34,SDSS CAND. AGN; MLCS2K2
1,RN,2,1,SDSS-II SN 13651,42.10,0.21,2110,SNIa SDSS,2014arXiv1401.3317S,SDSS-II SN 13651,0.249917,70,NaN,34,SDSS CAND. AGN; SALT 2
2,RN,999999,1,SDSS-II SN 13651,41.64,0.17,1700,SNIa SDSS,2014arXiv1401.3317S,SDSS-II SN 13651,0.249917,70,NaN,34,"SDSS-II SN 13651, SDSS CAND. AGN; MLCS2K2"
3,RN,999999,1,SDSS-II SN 13651,42.10,0.21,2110,SNIa SDSS,2014arXiv1401.3317S,SDSS-II SN 13651,0.249917,70,NaN,34,"SDSS-II SN 13651, SDSS CAND. AGN; SALT 2"
4,NaN,3,2,UGC 12889,34.16,0.40,68,Tully-Fisher,2007A&A...465...71T,NaN,NaN,NaN,NaN,27,mean


In [4]:
# find sources with NaN values for redshift, those are the ones we're interested in
# create a list with those names
listrs=np.unique(df['Galaxy ID'][np.isnan(df['redshift (z)'])])

In [5]:
from astroquery.ned import Ned

In [154]:
result_table.columns

<TableColumns names=('No.','Object Name','RA(deg)','DEC(deg)','Type','Velocity','Redshift','Redshift Flag','Magnitude and Filter','Distance (arcmin)','References','Notes','Photometry Points','Positions','Redshift Points','Diameter Points','Associations')>

In [63]:
# Here we request z, RA, DEC data from NED for each source in listrs
# this part skips errors due to NED not recognizing some names in listrs (weird)
# this will take > 2 hours
names=[]
rshifts=[]
RAlist=[]
DEClist=[]
k=0
for i in listrs[]:
    k=k+1
    try:
        result_table = Ned.query_object(i)
        if(not(result_table['Redshift'][0] is np.ma.masked)):
            if(np.mod(k,100)==0):
                print k,i, result_table['Redshift'][0],result_table['RA(deg)'][0],result_table['DEC(deg)'][0]
            names.append(i)
            rshifts.append(result_table['Redshift'][0])
            RAlist.append(result_table['RA(deg)'][0])
            DEClist.append(result_table['DEC(deg)'][0])
    except:
        pass

100 AGC 211465 0.035581 170.08417 34.30519
200 AGC 749315 0.002151 157.28375 26.90417
400 ARK 459 0.029494 221.61538 11.50735
500 Bol 225 -0.00055 11.12421 41.35704
600 CGCG 014-054 0.003686 187.7659 1.6757
700 CGCG 059-051 0.019654 122.86277 8.94103
800 CGCG 073-051 0.003889 204.18262 8.54688
900 CGCG 080-042 0.017152 247.43378 11.84712
1000 CGCG 121-025 0.038568 137.34293 24.24539
1100 CGCG 161-130 0.04405 205.7026 29.70575
1200 CGCG 313-029 0.022538 159.3833 65.01828
1300 CGCG 406-042 0.011888 349.27304 7.12283
1400 CGCG 452-010 0.02953 336.58192 19.58189
1500 CGCG 497-003 0.031105 350.00504 32.92286
1600 DDO 032 0.007397 48.65967 -4.77392
1800 ESO 027- G 017 0.026685 342.99779 -80.83583
1900 ESO 059- G 024 0.028073 120.61504 -72.42492
2000 ESO 091- G 010 0.015217 140.86721 -62.88225
2100 ESO 121- G 006 0.004039 91.87442 -61.80758
2200 ESO 148- G 020 0.010696 354.0864 -57.62934
2300 ESO 185- G 005 0.045165 294.56496 -52.82108
2400 ESO 206- G 019 0.036875 100.339 -51.41275
2500 ESO 2

In [86]:
# set up dataframe
# this is usually done by defining a dictionary-type object with our lists
diction={'Names' : names, 'Redshift': rshifts, 'RA': RAlist, 'DEC': DEClist}

In [87]:
df5=pd.DataFrame(diction)

In [100]:
# rearrange column order because for some reason the dictionary-to-dataframe step doesn't preserve order
cols=['Names', 'RA','DEC', 'Redshift']

In [102]:
df5=df5[cols]

In [103]:
df5.to_csv('redshiftdata.csv')

In [95]:
# no need for this, but it's good to know
df5=pd.concat([df3,df4])
df5.index = np.arange(len(df5))